# SPO2IDA

This methodology uses the SPO2IDA tool described in [Vamvatsikos and Cornell (2006)](http://onlinelibrary.wiley.com/doi/10.1002/eqe.573/abstract) to convert static pushover curves into $16\%$, $50\%$, and $84\%$ IDA curves. The SPO2IDA tool is based on empirical relationships obtained from a large database of incremental dynamic analysis results. This procedure is applicable to any kind of multi-linear capacity curve and it is suitable for single-building fragility curve estimation. Individual fragility curves can later be combined into a single fragility curve that considers the inter-building uncertainty. The figure below illustrates the IDA curves estimated using this methodology for a given capacity curve.

<img src="../../../../../figures/spo2ida.jpg" width="500" align="middle">

**Note**: To run the code in a cell:

1. Click on the cell to select it.
2. Press `SHIFT+ENTER` on your keyboard or press the play button (<button class='fa fa-play icon-play btn btn-xs btn-default'></button>) in the toolbar above.

In [ ]:
from rmtk.vulnerability.derivation_fragility.R_mu_T_dispersion.SPO2IDA import SPO2IDA_procedure 
from rmtk.vulnerability.common import utils
%matplotlib inline 

---
### Load capacity curves

In order to use this methodology, it is necessary to provide one (or a group) of capacity curves, defined according to the format described in the [RMTK manual](../../../../../rmtk-docs.pdf). In case multiple capacity curves are input, a spectral shape also needs to be defined.

1. Please provide the location of the file containing the capacity curves using the parameter `capacity_curves_file`.
2. Please also provide a spectral shape using the parameter `input_spectrum` if multiple capacity curves are used.

In [ ]:
capacity_curves_file = "../../../../../../rmtk_data/capacity_curves_Vb-dfloor.csv"
input_spectrum = "../../../../../../rmtk_data/FEMAP965spectrum.txt"

In [ ]:
capacity_curves = utils.read_capacity_curves(capacity_curves_file)
Sa_ratios = utils.get_spectral_ratios(capacity_curves, input_spectrum)
utils.plot_capacity_curves(capacity_curves)

### Idealise pushover curves

In order to use this methodology the pushover curves need to be idealised. Please choose an idealised shape using the parameter `idealised_type`. The valid options for this methodology are "bilinear" and "quadrilinear". Idealised curves can also be directly provided as input by setting the field `Idealised` to `TRUE` in the input file defining the capacity curves.

In [ ]:
idealised_type = "quadrilinear"

In [ ]:
idealised_capacity = utils.idealisation(idealised_type, capacity_curves)
utils.plot_idealised_capacity(idealised_capacity, capacity_curves, idealised_type)

### Load damage state thresholds

Please provide the path to your damage model file using the parameter `damage_model_file` in the cell below. Currently only `interstorey drift` damage model type is supported.

In [ ]:
damage_model_file = "../../../../../../rmtk_data/damage_model_ISD.csv"

In [ ]:
damage_model = utils.read_damage_model(damage_model_file)

### Calculate fragility functions

The damage threshold dispersion is calculated and integrated with the record-to-record dispersion through Monte Carlo simulations. Please enter the number of Monte Carlo samples to be performed using the parameter `montecarlo_samples` in the cell below.

In [ ]:
montecarlo_samples = 50

In [ ]:
fragility_model = SPO2IDA_procedure.calculate_fragility(capacity_curves, idealised_capacity, damage_model, montecarlo_samples, Sa_ratios, 1)

### Plot fragility functions

The following parameters need to be defined in the cell below in order to plot the lognormal CDF fragility curves obtained above:
* `minIML` and `maxIML`: These parameters define the limits of the intensity measure level for plotting the functions

In [ ]:
minIML, maxIML = 0.01, 2

In [ ]:
utils.plot_fragility_model(fragility_model, minIML, maxIML)

In [ ]:
print fragility_model

### Save fragility functions

The derived parametric fragility functions can be saved to a file in either CSV format or in the NRML format that is used by all OpenQuake input models. The following parameters need to be defined in the cell below in order to save the lognormal CDF fragility curves obtained above:
1. `taxonomy`: This parameter specifies a taxonomy string for the the fragility functions.
2. `minIML` and `maxIML`: These parameters define the bounds of applicability of the functions.
3. `output_type`: This parameter specifies the file format to be used for saving the functions. Currently, the formats supported are "csv" and "nrml".

In [ ]:
taxonomy = "RC"
minIML, maxIML = 0.01, 2.00
output_type = "csv"
output_path = "../../../../../../rmtk_data/output/"

In [ ]:
utils.save_mean_fragility(taxonomy, fragility_model, minIML, maxIML, output_type, output_path)

### Obtain vulnerability function

A vulnerability model can be derived by combining the set of fragility functions obtained above with a consequence model. In this process, the fractions of buildings in each damage state are multiplied by the associated damage ratio from the consequence model, in order to obtain a distribution of loss ratio for each intensity measure level. 

The following parameters need to be defined in the cell below in order to calculate vulnerability functions using the above derived fragility functions:
1. `cons_model_file`: This parameter specifies the path of the consequence model file.
2. `imls`: This parameter specifies a list of intensity measure levels in increasing order at which the distribution of loss ratios are required to be calculated.
3. `distribution_type`: This parameter specifies the type of distribution to be used for calculating the vulnerability function. The distribution types currently supported are "lognormal", "beta", and "PMF".

In [ ]:
cons_model_file = "../../../../../../rmtk_data/cons_model.csv"
imls = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 
        0.60, 0.70, 0.80, 0.90, 1.00, 1.20, 1.40, 1.60, 1.80, 2.00]
distribution_type = "lognormal"

In [ ]:
cons_model = utils.read_consequence_model(cons_model_file)
vulnerability_model = utils.convert_fragility_vulnerability(fragility_model, cons_model, 
                                                            imls, distribution_type)

### Plot vulnerability function



In [ ]:
utils.plot_vulnerability_model(vulnerability_model)

### Save vulnerability function

The derived parametric or nonparametric vulnerability function can be saved to a file in either CSV format or in the NRML format that is used by all OpenQuake input models. The following parameters need to be defined in the cell below in order to save the lognormal CDF fragility curves obtained above:
1. `taxonomy`: This parameter specifies a taxonomy string for the the fragility functions.
3. `output_type`: This parameter specifies the file format to be used for saving the functions. Currently, the formats supported are "csv" and "nrml".

In [ ]:
taxonomy = "RC"
output_type = "nrml"
output_path = "../../../../../../rmtk_data/output/"

In [ ]:
utils.save_vulnerability(taxonomy, vulnerability_model, output_type, output_path)